## Load MNIST on Python 3.x

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pickle
from scipy.ndimage import imread
import numpy as np

In [3]:
with open('./mnist.pkl', 'rb') as f:
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')

In [5]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

/home/wolterlw/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from keras.utils import to_categorical
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, Concatenate, Dropout, Flatten

In [8]:
def dense_block(filters=32, kernel_size=3):
    def f(x):
        x0 = BatchNormalization()(x)
        x0 = ReLU()(x0)
        x0 = Conv2D(filters, kernel_size=3, padding='same')(x0)
        x0 = Dropout(0.2)(x0)
        x0 = BatchNormalization()(x0)
        x0 = ReLU()(x0)
        x = Concatenate()([x, x0])
        return x
    return f
    
def bottleneck(filters=32):
    def f(x):
        x = BatchNormalization()(x)
        x = Conv2D(filters, kernel_size=1, padding='same')(x)
        x = MaxPool2D()(x)
        return x
    return f

In [12]:
inp = Input(shape=(28, 28,1))

x = dense_block(64, 3)(inp)
x = bottleneck(32)(x)
x = dense_block(32, 3)(x)
x = bottleneck(16)(x)
x = dense_block(16, 3)(x)
x = bottleneck(32)(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(10, activation='softmax')(x)

In [13]:
model = Model(inputs=[inp], outputs=[x])

In [14]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
Xtr = training_data[0].reshape(-1, 28, 28, 1)
ytr = to_categorical(training_data[1])

In [18]:
model.fit(Xtr, ytr, batch_size=128)

Epoch 1/1
50000/50000 [==============================] - 671s 13ms/step - loss: 0.2040 - acc: 0.9375
